In [1]:
 # one time execution
import nltk
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip
nltk.download('punkt')
nltk.download('stopwords')
!pip install 'scipy == 1.8.0'

--2022-12-13 06:42:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-12-13 06:42:18--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-12-13 06:42:19--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.4’

gl

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import nltk
import re

In [3]:
df = pd.read_csv("Test_intent.csv")

#print(df)

#print(df.keys())
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['responses']: # modify here to summarize intent/not intent
  sentences.append(sent_tokenize(s))

# sentences = [y for x in sentences for y in x]
# sentences = sentences[:7]
print(sentences[-1])

['Be more considerate of client having PTSD some kind of trauma.', 'Go back to appropriate self-care activities geared to me  Continue to make every effort to de-esclate using verbal skills.', 'Always operate from a trauma-informed practice  Using the "more trauma informed" skill before deciding on use of seclusion room.', 'Continue to provide debrief assessment to client even after they refuse.', 'Trauma informed care, de-escalation methods/approach, and least/last option/restraints  Trauma strategey  I will work harder to debrief with team and clients after an incident.', 'I will incorporate trauma awareness to my decision making in regards to applying restraints as a last resort.', 'Trauma informed care like talking clients through things, not staring, covering them Trauma-informed care and de-escalation  To assess risk of harm, engage the client and plan team intervention in violent emergent situations Trauma informed, self-care']


In [4]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
len(word_embeddings)

400000

In [5]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

sentence_vectors = []
for i in sentences:
    s = []
    for j in i:
        if len(j) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in j.split()])/(len(j.split())+0.001)
        else:
            v = np.zeros((100,))
        s.append(v)
    sentence_vectors.append(s)

In [6]:
sentence_vectors

[[array([-1.49831033e-01,  6.14757284e-02,  1.53550048e-01, -2.02756677e-01,
         -3.19919967e-02,  7.42800481e-02, -1.63639659e-01,  1.48315051e-01,
         -2.63200588e-01,  6.85680406e-02,  9.10607034e-02, -1.01018064e-01,
          3.73880530e-01, -3.09491790e-02, -6.92621069e-02, -3.14937853e-01,
          2.63706882e-01,  1.20916534e-01, -3.76298698e-01,  9.51569179e-02,
          9.54222763e-02, -5.71498804e-02,  1.91894797e-01, -2.42075553e-02,
          3.72459612e-02, -1.71053089e-01, -9.60836154e-02, -2.94227112e-01,
          8.02303076e-03, -9.11863551e-02, -1.33577992e-01,  4.36627468e-01,
         -2.02427634e-01,  7.62414359e-02,  6.23992811e-02,  1.62229202e-01,
         -5.15022894e-02,  3.05388318e-01, -1.11169776e-01, -9.60399998e-02,
         -4.26499984e-01, -1.94084436e-01,  2.67195433e-01, -1.83906413e-01,
          3.42742564e-02,  7.42471394e-02,  1.10297372e-01, -2.41760595e-01,
          2.13395318e-02, -6.19776769e-01,  8.11447573e-04, -1.04972374e-01,

In [7]:
# similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
big_sim_mat = []
for s in sentence_vectors:
    sim_mat = np.zeros([len(s), len(s)])

    for i in range(len(s)):
        for j in range(len(s)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(s[i].reshape(1,100), s[j].reshape(1,100))[0,0]
    big_sim_mat.append(sim_mat)

big_sim_mat

[array([[0.        , 0.95420806, 0.87254267, 0.90598762, 0.94984001,
         0.87165818],
        [0.95420806, 0.        , 0.88135833, 0.89842502, 0.91945938,
         0.83388161],
        [0.87254267, 0.88135833, 0.        , 0.86548458, 0.80264808,
         0.76269055],
        [0.90598762, 0.89842502, 0.86548458, 0.        , 0.87024805,
         0.82213576],
        [0.94984001, 0.91945938, 0.80264808, 0.87024805, 0.        ,
         0.87810063],
        [0.87165818, 0.83388161, 0.76269055, 0.82213576, 0.87810063,
         0.        ]]),
 array([[0.        , 0.76651297, 0.7381049 , 0.79504272, 0.79644082],
        [0.76651297, 0.        , 0.96395973, 0.95818501, 0.94542172],
        [0.7381049 , 0.96395973, 0.        , 0.92882543, 0.8999874 ],
        [0.79504272, 0.95818501, 0.92882543, 0.        , 0.94430956],
        [0.79644082, 0.94542172, 0.8999874 , 0.94430956, 0.        ]]),
 array([[0.        , 0.95754283, 0.89496544],
        [0.95754283, 0.        , 0.92335428],
        

In [8]:
import networkx as nx



In [9]:
!pip install rouge-score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:

from rouge_score import rouge_scorer
scores_list=[]
for i in range(5):
    nx_graph = nx.from_numpy_array(big_sim_mat[i])
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences[i])), reverse=True)
    gen = []
    ref = []
    for j in range (3):
        gen.append(ranked_sentences[j][1])
        sum = " ".join(gen)
    print(sum)
    ref = df['responses'].values[i]
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2','rougeL'], use_stemmer=True)
    scores = scorer.score(ref,sum)
    scores_list.append(scores)

    # a = 'Be more considerate of client having some kind of trauma. Always operate from a trauma-informed practice. De-esclate using verbal skills. Use the "more trauma informed" skill before deciding on use of seclusion room. Work harder to debrief with team and clients after an incident'
    # b = 'Be considerate of patient who has trauma. Using trauma-informed skills before using the seclusion room. Provide debrief assessment even if being refused to. Do trauma-informed care. Work harder to debrief client after trauma. Incorporate trauma awareness to decision on restraints. Assess rick of harm when dealing with client.'
    # scores = scorer.score(b,a)
    # print(scores)
# for j in range(len(sentences[-2])):
#     ref.append(sentences[-2][j])
# print(gen)
# print(ref)
# score = bleu(ref,gen)
# print(score)
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Work with other staff who already been training as team work To ensure we work safely as a team and be mindful of effect of our actions has on clients Safely manage agitated patient. Use technique to work with team Team work I plean to meet with my team and discuss strategies learnt. De-escalating  Trauma-informed care , team communication, use of restraints  Will use PINEL discontinuation strategies more effectively Clock manuveurs and more use of the Red Zone (awareness).


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Restraints use; Trauma informed practice (with examples) I will very mindful that trauma can make restraints event worst for patient [illegible] trauma informed awareness while restraint [illegible] 1) Remembering trauma-informed practice 2) Restraints-applying the correct way  The emergency use of mechanical restraints application and trauma aware strategies  I intend to use my awareness of different trauma informed strategies than using restraints. Being in the restraints allowed me to feel what it may be like for patients  Trauma informed care for restraints and thinking how being put in restraints can be traumatic. Now have hands on practice with restraints  Trauma-informed de-esclation prior to use of restraints  Applying restraints and trauma informed
If/when restraints needed, I feel confident applying  We have a client that needs to be restraint during ECT so this training gives me more confidence and knowledge how to do mechanical restraint Pin at the middle during restraint e

In [11]:
rouge1 = 0
rouge2 = 0
rougeL = 0
for s in scores_list:
    rouge1 += s['rouge1'][2]
    rouge2 += s['rouge2'][2]
    rougeL += s['rougeL'][2]


In [12]:
rouge1/len(scores_list)

0.8455079055913876

In [13]:
rouge2/len(scores_list)

0.8299212703030017

In [14]:
rougeL/len(scores_list)

0.7506094707746465